In [38]:
import torch
import numpy as np
import pandas as pd
import random
import torch.nn as nn
import torch.optim as optim
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [39]:
def fix_random(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

fix_random(42)

In [40]:
df = pd.read_csv("heart.csv")

In [41]:
features = df[['age', 'sex', 'trestbps', 'chol', 'thal', 'cp', 'exang']]
labels = df['target']

In [42]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [43]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [44]:
teacher = GradientBoostingClassifier()
teacher.fit(x_train_scaled, y_train)
soft_labels = teacher.predict_proba(x_train_scaled)

In [45]:
class StudentModel(nn.Module):
    def __init__(self, input_dim):
        super(StudentModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 2)
        )

    def forward(self, x):
        return self.net(x)

studentmodel = StudentModel(x_train.shape[1])

In [46]:
x_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_train.values, dtype=torch.long)
soft_tensor = torch.tensor(soft_labels, dtype=torch.float32)

criterion_ce = nn.CrossEntropyLoss()
criterion_kl = nn.KLDivLoss(reduction="batchmean")
optimizer = optim.Adam(studentmodel.parameters(), lr=0.001)

In [47]:
epochs = 50
alpha = 0.5  # blend between hard and soft labels

for epoch in range(epochs):
    studentmodel.train()
    optimizer.zero_grad()

    outputs = studentmodel(x_tensor)
    softmax_outputs = torch.log_softmax(outputs, dim=1)

    loss_soft = criterion_kl(softmax_outputs, soft_tensor)
    loss_hard = criterion_ce(outputs, y_tensor)
    loss = alpha * loss_hard + (1 - alpha) * loss_soft

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 0.5393
Epoch 10, Loss: 0.5227
Epoch 20, Loss: 0.5073
Epoch 30, Loss: 0.4925
Epoch 40, Loss: 0.4776


In [48]:
studentmodel.eval()
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)
with torch.no_grad():
    test_preds = torch.argmax(studentmodel(x_test_tensor), dim=1)

print("Accuracy on Test Data:", accuracy_score(y_test, test_preds))

Accuracy on Test Data: 0.6292682926829268


In [ ]:

def predict_heart_risk(input_list):
    input_np = np.array([input_list])
    scaled = scaler.transform(input_np)
    tensor = torch.tensor(scaled, dtype=torch.float32)

    studentmodel.eval()
    with torch.no_grad():
        output = studentmodel(tensor)
        pred = torch.argmax(output, dim=1).item()

    if pred == 1:
        print("\u2757 High Risk of Heart Attack — Consult a doctor immediately.")
    else:
        print("\u2705 No Significant Risk of Heart Attack — Keep maintaining a healthy lifestyle.")

predict_heart_risk([28, 1, 115, 170, 1, 1, 0])


❗ High Risk of Heart Attack — Consult a doctor immediately.


c:\Users\tharu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
